# Smoothed Heiken Ashi

In [1577]:
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta

In [1578]:
# get the data from last 60 days until the present
# dataF = yf.download("^NDX", start="2023-10-25", end="2023-12-21", interval='15m')
# dataF = yf.download("EURJPY=X", period="60d", interval='15m')
# dataF = yf.download("^NDX", period="365d", interval='1h')
# dataF = yf.download("EURJPY=X", period="50d", interval='30m') # <-- this is the one
# dataF = yf.download("EURJPY=X", period="365d", interval='1h')# <-- this is the one
dataF = yf.download("EURJPY=X", period="500d", interval='1h')# <--------- this is the one
# dataF = yf.download("EURUSD=X", period="365d", interval='1h') # XXXX Horrible
# dataF = yf.download("USDJPY=X", period="365d", interval='1h') <-- not too bad

# dataF.iloc[:,:]
#dataF.Open.iloc

[*********************100%%**********************]  1 of 1 completed


In [1579]:
df = pd.DataFrame(dataF)

# use df index, convert DateTime to  a column instead of index
df.reset_index(inplace=True)

# delete Adj Close
df = df.drop(['Adj Close'], axis=1)

# rename Datetime to "Gmt time"
df = df.rename(columns={'Datetime':'Gmt time'})

# set gmt time as index
df.set_index('Gmt time', inplace=True)

df

,Open,High,Low,Close,Volume
Gmt time,,,,,
2022-01-24 00:00:00+00:00,129.052002,129.098007,128.889999,129.078995,0
2022-01-24 01:00:00+00:00,129.080002,129.158005,128.990005,129.119995,0
2022-01-24 02:00:00+00:00,129.121002,129.143997,128.949997,128.983994,0
2022-01-24 03:00:00+00:00,128.987000,129.026993,128.929993,128.990997,0
2022-01-24 04:00:00+00:00,128.992004,129.072006,128.970001,129.057007,0
...,...,...,...,...,...
2023-12-22 18:00:00+00:00,156.927994,157.035004,156.878998,156.897003,0
2023-12-22 19:00:00+00:00,156.884995,156.951996,156.757004,156.910995,0
2023-12-22 20:00:00+00:00,156.910004,156.949997,156.822998,156.852997,0


In [1580]:
# get the 10 data point smoothed heiken ashi candles
df['HA_Close']=(df['Open']+ df['High']+ df['Low']+ df['Close'])/4
df['HA_Open']=(df['Open'].shift(1)+ df['Close'].shift(1))/2
df['HA_High']=df[['High','Open','Close']].max(axis=1)
df['HA_Low']=df[['Low','Open','Close']].min(axis=1)

# calculate the 10 period smoothed heiken ashi candles
df['HA_Close'] = df['HA_Close'].rolling(10).mean()
df['HA_Open'] = df['HA_Open'].rolling(10).mean()
df['HA_High'] = df['HA_High'].rolling(10).mean()
df['HA_Low'] = df['HA_Low'].rolling(10).mean()

# other indicators
df["ADX_14"]=ta.adx(df.High, df.Low, df.Close)['ADX_14']
df['RSI']=ta.rsi(df.Close, length=16)
df["20_MA"] = ta.ema(df.Close, length=20)
df["50_MA"] = ta.ema(df.Close, length=50)
df["100_MA"] = ta.ema(df.Close, length=100)
df["200_MA"]= ta.ema(df.Close, length=200)

In [1581]:
signal = [0]*len(df)

for row in range(1, len(df)):
  # isTradingTime = df.index[row].hour == 13 and df.index[row].minute == 0
  # if (not isTradingTime):
  #   continue
  
  current_heiken_ashi_candle_delta = df['HA_Close'].iloc[row] - df['HA_Open'].iloc[row]
  previous_heiken_ashi_candle_delta = df['HA_Close'].iloc[row-1] - df['HA_Open'].iloc[row-1]  

  if (current_heiken_ashi_candle_delta > 0 and previous_heiken_ashi_candle_delta < 0):
    # print(f"Buy at {df.iloc[row,0]}")
    hasSetup = True
    signal[row]=1
  elif (current_heiken_ashi_candle_delta < 0 and previous_heiken_ashi_candle_delta > 0):
    # print(f"Sell at {df.iloc[row,0]}")
    hasSetup = True
    signal[row]=-1
  else:
    hasSetup = False
    signal[row]=0

df['signal'] = signal


In [1582]:
def TotalSignal(l):
    isRanging = df.ADX_14[l] < 25
    if (
        df.signal[l]==1
        # and df.ADX_14[l]>25
        and df.RSI[l] > 45
        # and df.RSI[l] > 50
        and (not isRanging)
        # and df['50_MA'][l] < df.Close[l]
    ):
            return 2
    if (
        df.signal[l]==-1
        # and df.ADX_14[l]>25
        and df.RSI[l] < 55
        # and df.RSI[l] < 50
        and (not isRanging)
        # and df['50_MA'][l] > df.Close[l]
    ):
            return 1
    return 0
        
TotSignal = [0]*len(df)
for row in range(0, len(df)): #careful backcandles used previous cell
    TotSignal[row] = TotalSignal(row)
df['TotalSignal'] = TotSignal

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_61921/2295713913.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_61921/2295713913.py:4: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_61921/2295713913.py:13: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ip

In [1583]:
df[df.TotalSignal!=0].count()

Open           334
High           334
Low            334
Close          334
Volume         334
HA_Close       334
HA_Open        334
HA_High        334
HA_Low         334
ADX_14         332
RSI            334
20_MA          333
50_MA          332
100_MA         332
200_MA         327
signal         334
TotalSignal    334
dtype: int64

In [1584]:
import numpy as np
def pointposbreak(x):
    if x['TotalSignal']==1:
        return x['High']+1e-4
    elif x['TotalSignal']==2:
        return x['Low']-1e-4
    else:
        return np.nan

df['pointposbreak'] = df.apply(lambda row: pointposbreak(row), axis=1)

In [1585]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st=0
dfpl = df[st:st+ len(df)]
dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                # go.Scatter(x=dfpl.index, y=dfpl.VWAP, 
                #            line=dict(color='blue', width=1), 
                #            name="VWAP"), 
                # go.Scatter(x=dfpl.index, y=dfpl['BBL.0'], 
                #            line=dict(color='green', width=1), 
                #            name="BBL"),
                # go.Scatter(x=dfpl.index, y=dfpl['BBU.0'], 
                #            line=dict(color='green', width=1), 
                #            name="BBU")
                go.Scatter(x=dfpl.index, y=dfpl['50_MA'],
                            line=dict(color='red', width=1), 
                            name="50_MA"),
      ])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=10, color="MediumPurple"),
                name="Signal")

# set figure size
fig.update_layout(
    autosize=False,
    width=1600,
    height=1000,
    margin=dict(l=50,r=50,b=100,t=100,pad=4)
)

fig.show()

In [1586]:
dfpl = df[:75000].copy()
import pandas_ta as ta
dfpl['ATR']=ta.atr(dfpl.High, dfpl.Low, dfpl.Close, length=7)
#help(ta.atr)
def SIGNAL():
    return dfpl.TotalSignal

In [1587]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.08
    mysize = initsize
    dollarRisk = 50
    risk = 0
    latestEntry = 0
    
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 3*self.data.ATR[-1]
        TPSLRatio = 2
        
        # if the time is between 19:00 and 1:00 the next day, don't trade
        outsideTradingHour = (self.data.index[-1].hour >= 19) or (self.data.index[-1].hour < 1)
        # buy
        if ((not outsideTradingHour) and self.signal1==2):
            entry = self.data.Close[-1]
            # sl1 = entry - self.dollarRisk
            sl1 = entry - slatr
            tp1 = entry + TPSLRatio * slatr
            self.risk = entry - sl1
            
            # Close the sell position, if exist
            print(entry)
            print(slatr)
            self.position.close()
            print ("Close sell position", self.data.Close[-1])
            print("------")
            
            
            # Open a buy position
            self.buy(sl=sl1, tp=tp1, size=self.mysize, limit=entry)
            print("Buy", self.data.Close[-1])
            print("Stop loss: ", sl1)
            print("------")
            
            self.latestEntry = entry
        
        # sell
        elif ((not outsideTradingHour) and self.signal1==1):   
            entry = self.data.Close[-1]    
            # sl1 = entry + self.dollarRisk
            sl1 = entry + slatr
            tp1 = entry - TPSLRatio * slatr
            self.risk = sl1 - entry
            
            # Close the buy position, if exist
            self.position.close()
            print ("Close buy position", self.data.Close[-1])
            print("------")
            
            # place a sell limit order
            self.sell(sl=sl1, tp=tp1, size=self.mysize, limit=entry)
            print("Sell", self.data.Close[-1])
            print("Stop loss: ", sl1)
            print("------")
            
            self.latestEntry = entry
            
        # if the pl is greater than the stop loss amount, then move the stop loss to entry
        if (self.position.pl > self.risk):
            # adjust the stop loss to entry
            self.position.sl = self.latestEntry
            
        # close the position if the time is 19:00
        if (self.data.index[-1].hour == 19 and self.data.index[-1].minute == 0 and self.position.pl > 0 ):
            if (len(self.trades)>0):
                self.position.close()
                print("Close position due to time")
                print("------")
            
        # close the position if the amount lost exceeds 3% of the account balance
        if (self.position.pl < -1 * self.equity * 0.025):
            self.position.close()
            print("Close position due to loss")
            print("------")
        
        # # close position due to time
        # if (df.index[row].hour == 19 and df.index[row].minute == 0):
        #     if (len(self.trades)>0):
        #         self.position.close()
        #         print("Close position due to time")
        #         print("------")
        
        
            
        

In [1588]:
bt = Backtest(dfpl, MyStrat, cash=50000, margin=1/100, commission=0.00)
stat = bt.run()

128.86900329589844
0.7513132323587408
Close sell position 128.86900329589844
------
Buy 128.86900329589844
Stop loss:  128.1176900635397
------
Close buy position 128.7429962158203
------
Sell 128.7429962158203
Stop loss:  129.44524824887318
------
Close position due to time
------
128.63999938964844
0.5233623554894629
Close sell position 128.63999938964844
------
Buy 128.63999938964844
Stop loss:  128.11663703415897
------
Close buy position 128.7949981689453
------
Sell 128.7949981689453
Stop loss:  129.3725785486492
------
129.11000061035156
0.6701591896000226
Close sell position 129.11000061035156
------
Buy 129.11000061035156
Stop loss:  128.43984142075155
------
Close position due to time
------
129.4600067138672
0.5260731440096369
Close sell position 129.4600067138672
------
Buy 129.4600067138672
Stop loss:  128.93393356985754
------
Close buy position 129.14999389648438
------
Sell 129.14999389648438
Stop loss:  129.74300314903465
------
Close position due to time
------
Close 

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/backtesting/backtesting.py:952: UserWarning:

(2023-04-25 08:00:00+01:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119



In [1589]:
print(stat)

Start                     2022-01-24 00:00...
End                       2023-12-22 22:00...
Duration                    697 days 22:00:00
Exposure Time [%]                   20.672269
Equity Final [$]                 81743.298922
Equity Peak [$]                  96369.168335
Return [%]                          63.486598
Buy & Hold Return [%]               21.536433
Return (Ann.) [%]                   27.174213
Volatility (Ann.) [%]               54.286143
Sharpe Ratio                         0.500574
Sortino Ratio                         1.09883
Calmar Ratio                         1.139333
Max. Drawdown [%]                  -23.850992
Avg. Drawdown [%]                   -5.033382
Max. Drawdown Duration      174 days 05:00:00
Avg. Drawdown Duration       17 days 04:00:00
# Trades                                  275
Win Rate [%]                        50.545455
Best Trade [%]                       2.114028
Worst Trade [%]                     -1.058516
Avg. Trade [%]                    

In [1590]:
bt.plot(show_legend=False)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/backtesting/_plotting.py:122: UserWarning:

Data contains too many candlesticks to plot; downsampling to '2H'. See `Backtest.plot(resample=...)`



Row(id='119520', ...)